In [1]:
import scanpy as sc
data_dir = '../../dataset/merfish_human_neocortex_SSL.h5ad'
adata = sc.read_h5ad(data_dir)
adata.X = adata.raw.X.copy()
print(adata.X)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 12532157 stored elements and shape (234125, 300)>
  Coords	Values
  (0, 9)	5
  (0, 16)	2
  (0, 18)	3
  (0, 23)	1
  (0, 24)	1
  (0, 25)	5
  (0, 27)	1
  (0, 28)	1
  (0, 29)	6
  (0, 30)	1
  (0, 31)	1
  (0, 34)	6
  (0, 37)	1
  (0, 45)	21
  (0, 46)	1
  (0, 53)	1
  (0, 58)	1
  (0, 60)	1
  (0, 61)	1
  (0, 67)	2
  (0, 68)	1
  (0, 74)	1
  (0, 81)	1
  (0, 86)	1
  (0, 87)	2
  :	:
  (234123, 261)	1
  (234124, 14)	2
  (234124, 18)	1
  (234124, 25)	3
  (234124, 55)	1
  (234124, 58)	1
  (234124, 62)	1
  (234124, 66)	1
  (234124, 75)	1
  (234124, 89)	1
  (234124, 104)	1
  (234124, 106)	1
  (234124, 107)	1
  (234124, 117)	2
  (234124, 126)	1
  (234124, 146)	3
  (234124, 171)	1
  (234124, 184)	1
  (234124, 200)	1
  (234124, 215)	3
  (234124, 221)	1
  (234124, 237)	1
  (234124, 251)	1
  (234124, 261)	1
  (234124, 297)	1


In [2]:
import torch
import lightning.pytorch as pl
from self_supervision.models.lightning_modules.cellnet_autoencoder import MLPAutoEncoder
from self_supervision.estimator.cellnet import EstimatorAutoEncoder

# 设置你的 .ckpt 文件路径
ckpt_path = "../../sc_pretrained/Pretrained Models/GPMask.ckpt"

# 模型参数
units_encoder = [512, 512, 256, 256, 64]
units_decoder = [256, 256, 512, 512]

# 初始化 EstimatorAutoEncoder 实例
estim = EstimatorAutoEncoder(data_path=None)  # 如果没有实际数据路径，可以设置为None

# 加载预训练模型
estim.model = MLPAutoEncoder.load_from_checkpoint(
    ckpt_path,
    gene_dim=19331,  # 根据你的数据调整
    batch_size=2048,  # 根据你的需要调整
    units_encoder=units_encoder, 
    units_decoder=units_decoder,
    masking_strategy="random",  # 假设模型使用了随机掩码
    masking_rate=0.5,  # 根据需要调整
)

# 使用 GPU 进行评估（如果可用）
estim.trainer = pl.Trainer(accelerator="gpu", devices=1 if torch.cuda.is_available() else None)
estim.model

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs


MLPAutoEncoder(
  (train_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=test_
  )
  (encoder): MLP(
    (0): Linear(in_features=19331, out_features=512, bias=True)
    (1): SELU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SELU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SELU()
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=256, out_features=256, b

In [3]:
import pandas as pd
var_df = pd.read_parquet('../../sc_pretrained/var.parquet')
var_df

,feature_id,feature_name
0,ENSG00000186092,OR4F5
1,ENSG00000284733,OR4F29
2,ENSG00000284662,OR4F16
3,ENSG00000187634,SAMD11
4,ENSG00000188976,NOC2L
...,...,...
19326,ENSG00000288702,UGT1A3
19327,ENSG00000288705,UGT1A5
19328,ENSG00000182484,WASH6P
19329,ENSG00000288622,PDCD6-AHRR


In [4]:
all_genes = var_df['feature_name'].tolist()
all_genes

['OR4F5',
 'OR4F29',
 'OR4F16',
 'SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'PERM1',
 'HES4',
 'ISG15',
 'AGRN',
 'RNF223',
 'C1orf159',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'C1QTNF12',
 'UBE2J2',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'INTS11',
 'CPTP',
 'TAS1R3',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20',
 'ANKRD65',
 'TMEM88B',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'TMEM240',
 'SSU72',
 'FNDC10',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'SLC35E2B',
 'CDK11A',
 'NADK',
 'GNB1',
 'CALML6',
 'TMEM52',
 'CFAP74',
 'GABRD',
 'PRKCZ',
 'FAAP20',
 'SKI',
 'MORN1',
 'RER1',
 'PEX10',
 'PLCH2',
 'PANK4',
 'HES5',
 'TNFRSF14',
 'PRXL2B',
 'MMEL1',
 'TTC34',
 'ACTRT2',
 'PRDM16',
 'ARHGEF16',
 'MEGF6',
 'TPRG1L',
 'WRAP73',
 'TP73',
 'CCDC27',
 'SMIM1',
 'LRRC47',
 'CEP104',
 'DFFB',
 'C1orf174',
 'AJAP1',
 'NPHP4',
 'KCNAB2',
 'CHD5',
 'RPL22',
 'RNF207',
 'ICMT',
 'HES3',
 'GPR153',
 'ACOT7',
 'HES2',
 'ESPN',
 'TNFRSF25',
 'PLEKHG5',
 'NOL9',
 'TAS1R1',
 'ZBTB48',
 'KLH

In [5]:
adata.var['gene_name']=adata.var.index
adata.var['gene_name']

features
TFAP2C          TFAP2C
HGF                HGF
NAMPT            NAMPT
VIM                VIM
CD38              CD38
               ...    
LINC01170    LINC01170
SATB1-AS1    SATB1-AS1
CFAP54          CFAP54
RMST              RMST
NOTCH1          NOTCH1
Name: gene_name, Length: 300, dtype: object

In [6]:
import numpy as np
# 初始化一个新的数据矩阵，形状为 (adata.X.shape[0], len(all_genes))，填充为零
new_data = np.zeros((adata.X.shape[0], len(all_genes)), dtype=np.float32)

In [7]:
existing_genes = adata.var['gene_name']
existing_genes

features
TFAP2C          TFAP2C
HGF                HGF
NAMPT            NAMPT
VIM                VIM
CD38              CD38
               ...    
LINC01170    LINC01170
SATB1-AS1    SATB1-AS1
CFAP54          CFAP54
RMST              RMST
NOTCH1          NOTCH1
Name: gene_name, Length: 300, dtype: object

In [8]:
# 将所有基因名称转换为小写
all_genes_lower = [gene.lower() for gene in all_genes]
adata_genes_lower = [gene.lower() for gene in existing_genes]

# 将两个列表转换为集合
all_genes_set = set(all_genes_lower)
adata_genes_set = set(adata_genes_lower)

# 计算交集
matching_genes = all_genes_set.intersection(adata_genes_set)
matching_count = len(matching_genes)
# 计算不匹配的基因
non_matching_genes = adata_genes_set - matching_genes
non_matching_count = len(non_matching_genes)


# 输出结果
print(f"匹配的基因数量: {matching_count}")
print(f"匹配的基因列表: {matching_genes}")
non_matching_genes


匹配的基因数量: 281
匹配的基因列表: {'pdzrn4', 'slco1c1', 'camk2a', 'znf536', 'carhsp1', 'cldn5', 'prex2', 'olig1', 'aqp4', 'adarb2', 'il1rapl2', 'nhlh2', 'tcf7l2', 'ebf3', 'brip1', 'znf385d', 'cadps2', 'eogt', 'notch1', 'pou3f2', 'itih5', 'rorb', 'aspm', 'znf804b', 'stk32b', 'ttr', 'hgf', 'c21orf58', 'cntln', 'gad1', 'ralyl', 'cntnap4', 'slc9a9', 'klhl14', 'id3', 'adamtsl1', 'mobp', 'nxph1', 'cfap299', 'maml2', 'plp1', 'creb5', 'nefl', 'runx1t1', 'pcdh15', 'tnr', 'rnf220', 'kcnip4', 'fam160a1', 'hcrtr2', 'pou6f2', 'fezf2', 'cped1', 'itga2', 'clmp', 'cfap54', 'prss12', 'fbn2', 'opcml', 'hmgb2', 'col1a2', 'ebf1', 'vwc2l', 'dscam', 'arx', 'kcnq5', 'sv2b', 'cebpd', 'tnc', 'palmd', 'cryab', 'eomes', 'ascl1', 'dock8', 'hes1', 'gja1', 'pcdh11y', 'kcnh5', 'ifit3', 'mdga1', 'rbms3', 'slit3', 'thsd4', 'ifi6', 'neurod1', 'apold1', 'chrdl1', 'moxd1', 'kirrel3', 'olfm3', 'hes6', 'nxph2', 'cobll1', 'dach1', 'dock10', 'pbx3', 'nrp1', 'kif11', 'cpne4', 'sulf1', 'igfbp5', 'cbln2', 'col4a1', 'sulf2', 'cdh9', 'sema5a

{'dlx6-as1',
 'fam106a',
 'kcnq1ot1',
 'lhx1-dt',
 'linc00609',
 'linc01170',
 'linc01619',
 'linc01965',
 'linc02147',
 'linc02487',
 'lncarod',
 'neat1',
 'nfia-as2',
 'nr2f2-as1',
 'obi1-as1',
 'rmst',
 'satb1-as1',
 'sgo1-as1',
 'tctex1d1'}

In [9]:
gene_to_index = {gene: idx for idx, gene in enumerate(all_genes_lower)}
gene_to_index

{'or4f5': 0,
 'or4f29': 1,
 'or4f16': 2,
 'samd11': 3,
 'noc2l': 4,
 'klhl17': 5,
 'plekhn1': 6,
 'perm1': 7,
 'hes4': 8,
 'isg15': 9,
 'agrn': 10,
 'rnf223': 11,
 'c1orf159': 12,
 'ttll10': 13,
 'tnfrsf18': 14,
 'tnfrsf4': 15,
 'sdf4': 16,
 'b3galt6': 17,
 'c1qtnf12': 18,
 'ube2j2': 19,
 'scnn1d': 20,
 'acap3': 21,
 'pusl1': 22,
 'ints11': 23,
 'cptp': 24,
 'tas1r3': 25,
 'dvl1': 26,
 'mxra8': 27,
 'aurkaip1': 28,
 'ccnl2': 29,
 'mrpl20': 30,
 'ankrd65': 31,
 'tmem88b': 32,
 'vwa1': 33,
 'atad3c': 34,
 'atad3b': 35,
 'atad3a': 36,
 'tmem240': 37,
 'ssu72': 38,
 'fndc10': 39,
 'mib2': 40,
 'mmp23b': 41,
 'cdk11b': 42,
 'slc35e2b': 43,
 'cdk11a': 44,
 'nadk': 45,
 'gnb1': 46,
 'calml6': 47,
 'tmem52': 48,
 'cfap74': 49,
 'gabrd': 50,
 'prkcz': 51,
 'faap20': 52,
 'ski': 53,
 'morn1': 54,
 'rer1': 55,
 'pex10': 56,
 'plch2': 57,
 'pank4': 58,
 'hes5': 59,
 'tnfrsf14': 60,
 'prxl2b': 61,
 'mmel1': 62,
 'ttc34': 63,
 'actrt2': 64,
 'prdm16': 65,
 'arhgef16': 66,
 'megf6': 67,
 'tprg1l': 68

In [10]:
only_in_all_genes = all_genes_set - adata_genes_set

only_in_adata_genes = adata_genes_set - all_genes_set

# 输出结果
print(f"仅在 all_genes 中存在的基因数量: {len(only_in_all_genes)}")
print(f"仅在 all_genes 中存在的基因: {only_in_all_genes}")

print(f"仅在 adata_genes 中存在的基因数量: {len(only_in_adata_genes)}")
print(f"仅在 adata_genes 中存在的基因: {only_in_adata_genes}")


仅在 all_genes 中存在的基因数量: 19050
仅在 all_genes 中存在的基因: {'pth1r', 'pramef5', 'or4m2b', 'ehmt2', 'clps', 'psmb2', 'pou4f3', 'osbp', 'tspan33', 'bub3', 'wdr5b', 'ct47a11', 'stab2', 'pcdhb1', 'cmas', 'apobec3b', 'h2ac8', 'plekhb1', 'zmym2', 'vps13b', 'plppr2', 'ykt6', 'dnmbp', 'smim7', 'sdsl', 'rpl35', 'rpl36a', 'slc41a1', 'spaca6', 'aif1', 'tmprss11b', 'rxfp2', 'rreb1', 'rps5', 'fam83c', 'prxl2b', 'bcl6b', 'c21orf91', 'spred1', 'smurf2', 'fbxl22', 'lim2', 'tbc1d3l', 'ess2', 'rock1', 'tpgs2', 'yjefn3', 'or56b1', 'abcb10', 'arhgap9', 'tcf20', 'eif4g1', 'trappc14', 'tbc1d9', 'armcx4', 'nobox', 'twnk', 'cntf', 'shkbp1', 'c9orf24', 'irx3', 'faim2', 'c1r', 'nfic', 'gtf2e2', 'eml1', 'zc3h8', 'dnah3', 'or52l1', 'ncapd3', 'agrp', 'ppial4a', 'armcx1', 'znf584', 'gimap7', 'smug1', 'atf7ip', 'mfge8', 'itpkb', 'pdcd6', 'ier3', 'zbtb22', 'ly6h', 'muc20', 'frmd5', 'palm', 'svep1', 'alpl', 'epc2', 'crym', 'pcdhga10', 'ccdc74b', 'rab39a', 'cpt1a', 'acvr1', 'or11g2', 'c6orf47', 'spata22', 'dennd1b', 'pik3c3', '

In [11]:
dense_adata_X = adata.X.toarray()
for i, gene in enumerate(adata_genes_lower):
    if gene in gene_to_index:
        new_data[:, gene_to_index[gene]] = dense_adata_X[:, i]
    else:
        print(f'Gene {gene} not found in all_genes list')

Gene tctex1d1 not found in all_genes list


Gene linc02487 not found in all_genes list
Gene nfia-as2 not found in all_genes list
Gene lncarod not found in all_genes list
Gene dlx6-as1 not found in all_genes list
Gene sgo1-as1 not found in all_genes list


Gene linc02147 not found in all_genes list
Gene linc01965 not found in all_genes list
Gene linc00609 not found in all_genes list
Gene linc01619 not found in all_genes list
Gene kcnq1ot1 not found in all_genes list
Gene obi1-as1 not found in all_genes list
Gene fam106a not found in all_genes list
Gene neat1 not found in all_genes list
Gene nr2f2-as1 not found in all_genes list
Gene lhx1-dt not found in all_genes list
Gene linc01170 not found in all_genes list
Gene satb1-as1 not found in all_genes list
Gene rmst not found in all_genes list


In [12]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split


label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(adata.obs['cell_type'])  # 预先编码标签


random_seed = 42
X_train_val, X_test, y_train_val, y_test = train_test_split(
    new_data, labels_encoded, test_size=0.15, random_state=random_seed)


X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, random_state=random_seed)  # 0.1765 是为了让验证集占 15%

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

estim.model.eval()
with torch.no_grad():
    X_train_tensor = torch.tensor(X_train).float().to(device)
    X_test_tensor = torch.tensor(X_test).float().to(device)
    train_embeddings = estim.model.encoder(X_train_tensor).detach().cpu().numpy()
    test_embeddings = estim.model.encoder(X_test_tensor).detach().cpu().numpy()


cuda


In [13]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

    

    # 初始化和训练KNN分类器
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_embeddings, y_train)
    
    # 模型预测
predictions = knn.predict(test_embeddings)

    # 计算准确率和 F1 分数
accuracy = accuracy_score(y_test, predictions)
print(f"KNN Accuracy on Test Data: {accuracy}")
f1 = f1_score(y_test, predictions, average='weighted')
print(f"Weighted F1 Score: {f1}")
    
macro_f1 = f1_score(y_test, predictions, average='macro')
print(f'Macro F1 Score: {macro_f1}')

    # 计算随机猜测的准确率
class_probabilities = np.bincount(y_test) / len(y_test)
random_accuracy = np.sum(class_probabilities ** 2)
print(f"Random Guess Accuracy: {random_accuracy}")

    # 生成分类报告
report = classification_report(y_test, predictions, target_names=label_encoder.classes_)
print(report)

KNN Accuracy on Test Data: 0.6859249978644039
Weighted F1 Score: 0.6813572163867421
Macro F1 Score: 0.6506708763273837
Random Guess Accuracy: 0.05408382924188506
                          precision    recall  f1-score   support

       Astrocyte-Fibrous       0.62      0.84      0.72      2056
  Astrocyte-Protoplasmic       0.59      0.59      0.59      1958
          EN-IT-Immature       0.67      0.72      0.69      3827
              EN-L2_3-IT       0.65      0.75      0.69      2843
                EN-L4-IT       0.68      0.67      0.68      2585
                EN-L5-ET       0.58      0.47      0.52        98
                EN-L5-IT       0.73      0.77      0.75      1670
              EN-L5_6-NP       0.78      0.69      0.73       344
                EN-L6-CT       0.67      0.69      0.68       778
                EN-L6-IT       0.76      0.78      0.77      1914
                  EN-L6b       0.83      0.62      0.71      1218
              EN-Newborn       0.74      0.56

In [14]:
with torch.no_grad():
    new_data_tensor = torch.tensor(new_data).float().to(device)
    SSL_embeddings = estim.model.encoder(new_data_tensor).detach().cpu().numpy()
new_adata = sc.read_h5ad(data_dir)
new_adata.obsm[f'SSL_GP_ZS_{random_seed}'] = SSL_embeddings
new_adata.uns[f'GP_ZS_y_test_{random_seed}'] = y_test
new_adata.uns[f'GP_ZS_predictions_{random_seed}'] = predictions
new_adata.uns[f'GP_ZS_target_names_{random_seed}'] = label_encoder.classes_
new_adata.write_h5ad(data_dir)

In [15]:

import pandas as pd
import os
import re

# 当前 Notebook 文件名
notebook_name = "MERSCOPE_human_neocortex_GP_mask_zero_shot_42.ipynb"

# 初始化需要打印的值
init_train_loss = train_losses[0] if 'train_losses' in globals() else None
init_val_loss = val_losses[0] if 'val_losses' in globals() else None
converged_epoch = len(train_losses) - patience if 'train_losses' in globals() else None
converged_val_loss = best_val_loss if 'best_val_loss' in globals() else None

# 打印所有所需的指标
print("Metrics Summary:")
if 'train_losses' in globals():
    print(f"init_train_loss\tinit_val_loss\tconverged_epoch\tconverged_val_loss\tmacro_f1\tweighted_f1\tmicor_f1")
    print(f"{init_train_loss:.3f}\t{init_val_loss:.3f}\t{converged_epoch}\t{converged_val_loss:.3f}\t{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")
else:
    print(f"macro_f1\tweighted_f1\tmicor_f1")
    print(f"{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")

# 保存结果到 CSV 文件
output_data = {
    'dataset_split_random_seed': [int(random_seed)],
    'dataset': ['MERSCOPE_human_neocortex'],
    'method': [re.search(r'neocortex_(.*?)_\d+', notebook_name).group(1)],
    'init_train_loss': [init_train_loss if init_train_loss is not None else ''],
    'init_val_loss': [init_val_loss if init_val_loss is not None else ''],
    'converged_epoch': [converged_epoch if converged_epoch is not None else ''],
    'converged_val_loss': [converged_val_loss if converged_val_loss is not None else ''],
    'macro_f1': [macro_f1],
    'weighted_f1': [f1],
    'micor_f1': [accuracy]
}
output_df = pd.DataFrame(output_data)

# 保存到当前目录下名为 results 的文件夹中
if not os.path.exists('results'):
    os.makedirs('results')

csv_filename = f"results/{os.path.splitext(notebook_name)[0]}_results.csv"
output_df.to_csv(csv_filename, index=False)


Metrics Summary:
macro_f1	weighted_f1	micor_f1
0.651	0.681	0.686
